# ちょっぴりDD-Athenaデモ

今回は説明・デモ全般をSageMaker Notebookで行ってみようと思います！

- Athena 基礎
- Athena Federated Query
- Athena UDF
- Athena EXPLAIN
- Athena ML

## Athena 基礎

Athena ではクエリのためにテーブル定義が必要で、デフォルトでは AWS Glue Data Catalog 上のテーブル定義を使用します。  
AWS Glue Data Catalog にテーブル定義を作成する方法は次の 3 つです。

- Athena DDL
- AWS Glue Catalog API
- AWS Glue Crawler

今回はAthena DDLでテーブルを作成してみましょう。
その前にデータベースをSDKを使って作成します。

データベース作成が完了したら[Athenaコンソール](https://console.aws.amazon.com/athena/home?region=us-east-1#query)で確認してみます。

1. S3にあるデータソースを覗いてみましょう
2. テーブル作成してみる

In [4]:
# 必要なものをimportしておく
import sys
!{sys.executable} -m pip install PyAthena
from pyathena import connect 
import pandas as pd
import sagemaker
import boto3

### 基礎1. S3にあるデータを覗いてみる

In [63]:
import pandas as pd
bucket = 'dddemo-067150986393'
csv_location = 's3://' + bucket + '/data/year=2021/monster.csv'

df = pd.read_csv(csv_location, header=None, names=['id', 'monster_name'])
df.head(10)

,id,monster_name
0,1,スライム
1,2,ドラキー
2,3,スライムベス
3,4,ゴースト
4,5,モーモン
5,6,しましまキャット
6,7,メーダ
7,8,いたずらもぐら
8,9,リリパット
9,10,バブルスライム


![](./assets/monster.png)

### 基礎2. テーブル作成してみる

csvで保存されたファイルを読み込んでテキスト形式のテーブルを作ってみる。まずはデータベース作成をします。

In [18]:
s3 = boto3.client('s3')
bucket = 'dddemo-067150986393'
db = 'dddemodb'
wg = 'dddemo'
output_location = 's3://' + bucket + '/athena-dd/'
connection = connect(s3_staging_dir=output_location, region_name='us-east-1', work_group=wg)

# データベース作成
q = '''
CREATE DATABASE {};
'''.format(db)

# pd.read_sql(q, connection)

q = '''
SHOW DATABASES;
'''
pd.read_sql(q, connection)

,database_name
0,dddemodb
1,default
2,mydb
3,qshandson
4,redis
5,sampledb


次にテーブルを作成しましょう。

Athena DDLの記法例

- Athena の DDL は HiveQL 形式で記述
- 標準的なテーブル定義ステートメントの後に、パーティション定義、データ形式、データの場所、圧縮形式などを指定


```
CREATE EXTERNAL TABLE IF NOT EXISTS action_log (
      user_id string,
      action_category string,
      action_detail string
)
  PARTITIONED BY (year int, month int)  -- パーディション
  STORED AS PARQUET -- データ形式
  LOCATION 's3://athena-examples/action-log/’ -- データの場所
  TBLPROPERTIES ('PARQUET.COMPRESS'='SNAPPY'); -- 圧縮形式

```

In [100]:
tablename_parquet = 'monster_parquet'
data_location = 's3://' + bucket + '/data/year=2021/' # 今回はパーティションはしない

q = """
CREATE EXTERNAL TABLE IF NOT EXISTS {}.{} (
id string,
monster_name string
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES ('serialization.format' = ',', 'field.delim' = ',')
STORED AS TEXTFILE
LOCATION '{}'
TBLPROPERTIES ('has_encrypted_data'='false',  'skip.header.line.count'='1')
; 
""".format(db, tablename_parquet, data_location)

pd.read_sql(q, connection)

q = """
MSCK REPAIR TABLE {}. {};
""".format(db, tablename_parquet)
pd.read_sql(q, connection)

""


In [101]:
# 作成したテーブルに対してクエリを実行してみる
q = """
SELECT * FROM {}.{};
""".format(db, tablename_parquet)

print(q)

pd.read_sql(q, connection)


SELECT * FROM dddemodb.monster_parquet;



,id,monster_name
0,1,スライム
1,2,ドラキー
2,3,スライムベス
3,4,ゴースト
4,5,モーモン
...,...,...
374,375,ハロウィンルイーダ
375,376,アバン
376,377,エデンの戦士たち１
377,378,エデンの戦士たち２


#### クエリ結果

実行される各クエリのクエリ結果とメタデータ情報は、指定した S3 バケットに自動的に保存されます。  
この保存自体をオフにすることはできません。  
[Athena コンソール履歴画面](https://console.aws.amazon.com/athena/query-history/home?region=us-east-1)から、クエリ結果ファイルを直接ダウンロードすることが可能です。

In [102]:
# もういらないのでテーブルdropしておく
q = """
DROP TABLE `{}.{}`;
""".format(db, tablename_parquet)

pd.read_sql(q, connection)

""


## Athena フェデレーテッドクエリ

様々なデータソースに対して SQL クエリを実行可能です。

AWS Lambda で動作するコネクタを利用して実行する仕組みです。

標準のコネクタとしてさまざまなデータソースが対応済となっています。
- Amazon DynamoDB
- Amazon Redshift
- Apache HBase
- MySQL
- PostgreSQL など

Athena Query Federation SDK を利用して、独自コネクタを実装可能です。


1. lambda作成

![](./assets/federated.png)

## フェデレーテッドクエリ1. lambda作成

今回はjupyter notebookのterminalを使ってインストールを行いますが、  
cloud9を用いた方がスクリプト実行で全て実施できるので導入がスムーズです。  
詳細は[こちら](https://github.com/awslabs/aws-athena-query-federation/tree/master/athena-jdbc)

事前に以下が済んでいることを前提とします。  
mavenをインストールするスクリプトがSageMaker Notebookだと使えないので手動でインストールしました。

- [samのインストール](https://docs.aws.amazon.com/ja_jp/serverless-application-model/latest/developerguide/serverless-sam-cli-install-linux.html) （cloud9の場合は最初から入っています）
- [AWS Serverless Applicationへのアクセス権限追加](https://docs.aws.amazon.com/ja_jp/serverlessrepo/latest/devguide/security_iam_id-based-policy-examples.html)

#### mavenインストール手順

tar.gzを任意のディレクトに配置してください。

```bash
sudo mv apache-maven-3.6.3 /opt/
cd /opt/
sudo ln -s /opt/apache-maven-3.6.3 apache-maven

cp ~/.bash_profile{,.bk}
echo 'MVN_HOME=/opt/apache-maven' >> ~/.bash_profile
echo 'PATH=$MVN_HOME/bin:$PATH:$HOME/.local/bin:$HOME/bin' >> ~/.bash_profile
source ~/.bash_profile

mvn --version

cd -
```

/bin/sh: /usr/local/bin/mvn: No such file or directory
